In [2]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
import time
from get_dataset_info import *

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [3]:
def estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px, alpha):
    
    err_threshold = err_threshold_px / K[0,0]
    best_inliers = None
    best_E = None
    max_inliers = 0
    epsilon = 0
    T = n_its
    n_points = x1_norm.shape[1]

    for t in trange(n_its):

        rand_mask = np.random.choice(np.size(x1_norm,1), n_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers
            print(np.sum(inliers), end='\r')

            epsilon = max_inliers / n_points
            T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
            print('New T:', T, 'New epsilon:', epsilon)
            if t >= T-1:
                print('Early bailout at iteration:', t, T)
                break
        
    return best_E, best_inliers

In [9]:
report = '/report-images'
data_set = 0
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]

In [8]:
marg = 0.7
x1_init, x2_init, img_match = cv.compute_sift_points(img1_init, img2_init, marg)
np.save('data/x1_init_{}.npy'.format(data_set), x1_init)
np.save('data/x2_init_{}.npy'.format(data_set), x2_init)

Number of matches: 44542
Number of good matches: 16542


In [ ]:
x1_init = np.load('data/x1_init_{}.npy'.format(data_set))
x2_init = np.load('data/x2_init_{}.npy'.format(data_set))
x1_init_norm = cv.transform_and_dehomogenize(K_inv, x1_init)
x2_init_norm = cv.transform_and_dehomogenize(K_inv, x2_init)
n_its = 100000
n_samples = 8
alpha = 0.95
estimate_E_robust(K, x1_init_norm, x2_init_norm, n_its, n_samples, pixel_threshold, alpha)



In [16]:
alpha = 0.5
epsilon = 0.2
s = 8
np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))

270761.0

In [12]:
plt_3D = True
save = False

x = np.linspace(0,10,11)
y = np.random.rand(11)
fig = plt.figure()
plt.plot(x,y)
fig.savefig('report-images/test.png', bbox_inches='tight')
img = cv.load_image('report-images/test.png')
plt.imshow(img)